# Probamos varios modelos de recomendación

# 1.

In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler



def recomendacion1(titulo):

    # Cargar datos dentro de la función
    movies_df = pd.read_parquet('../Datasets/movies.parquet')

    # Asegúrate de que la columna de título esté en minúsculas para la búsqueda
    titulo = titulo.lower()
    
    # Filtra el DataFrame para la película de entrada
    pelicula = movies_df[movies_df['title'].str.lower() == titulo]
    
    if pelicula.empty:
        return "Película no encontrada."
    
    # Preparar datos para vectorización
    # Crear el vectorizador para el título
    title_vectorizer = TfidfVectorizer(stop_words='english')
    # Crear el codificador para el género
    genre_encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    
    # Encajamos el codificador y el vectorizador
    title_vectorizer.fit(movies_df['title'])
    genre_encoder.fit(movies_df['genre_name'].apply(lambda x: x.split('|')).tolist())
    
    # Transformamos los datos
    title_vectors = title_vectorizer.transform(movies_df['title'])
    genre_vectors = genre_encoder.transform(movies_df['genre_name'].apply(lambda x: x.split('|')).tolist())
    popularity = movies_df['popularity'].values.reshape(-1, 1)
    
    # Normalizamos la popularidad
    scaler = StandardScaler()
    popularity = scaler.fit_transform(popularity)
    
    # Convertimos popularidad a una matriz dispersa
    popularity = csr_matrix(popularity)
    
    # Concatenamos todos los vectores
    combined_vectors = hstack([title_vectors, genre_vectors, popularity])
    
    # Encuentra el índice de la película de entrada
    idx = movies_df.index[movies_df['title'].str.lower() == titulo].tolist()[0]
    
    # Calcula la similitud de coseno
    cosine_sim = cosine_similarity(combined_vectors[idx:idx+1], combined_vectors).flatten()
    
    # Crea una lista de similitud excluyendo la película de entrada
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtén las 5 películas más similares
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    
    # Devuelve los nombres de las películas recomendadas
    recommended_movies = movies_df['title'].iloc[movie_indices].tolist()
    
    return recommended_movies

# Ejemplo de uso
# movies_df = pd.read_csv('path_to_movies_data.csv')
print(recomendacion('toy story'))



['Toy Story 2', 'Toy Story 3', 'Up', '9', 'Your Name.']


# 2.

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack



def recomendacion2(titulo, randomize=True, genre_weight=3):

    # Datos
    movies_df = pd.read_parquet('../Datasets/movies_df.parquet')

    # Agrupar los géneros por título para evitar duplicados
    movies_df_grouped = movies_df.groupby('title').agg({
    'genre_name': lambda x: list(set(x))
    }).reset_index()

    titulo = titulo.lower()
    
    pelicula = movies_df_grouped[movies_df_grouped['title'].str.lower() == titulo]
    
    if pelicula.empty:
        return "Película no encontrada."
    
    # Vectorizar los títulos
    title_vectorizer = TfidfVectorizer(stop_words='english')
    title_vectors = title_vectorizer.fit_transform(movies_df_grouped['title'])
    
    # Codificar los géneros
    genre_encoder = MultiLabelBinarizer()
    genre_vectors = genre_encoder.fit_transform(movies_df_grouped['genre_name'])
    
    # Ponderar más los géneros
    genre_vectors = genre_vectors * genre_weight
    
    # Combinar vectores de título y género
    combined_vectors = hstack([title_vectors, genre_vectors]).tocsr()
    
    # Obtener el índice de la película de entrada
    idx = movies_df_grouped.index[movies_df_grouped['title'].str.lower() == titulo].tolist()[0]
    
    # Calcular la similitud de coseno
    input_vector = combined_vectors[idx]
    cosine_sim = cosine_similarity(input_vector, combined_vectors).flatten()
    
    # Excluir la película de entrada y ordenar por similitud
    sim_scores = sorted(list(enumerate(cosine_sim)), key=lambda x: x[1], reverse=True)
    sim_scores = [score for score in sim_scores if score[0] != idx]
    
    # Seleccionar las películas recomendadas (máximo 5)
    recommended_titles = set()
    top_similar = [score for score in sim_scores if score[1] > 0.7]  # Filtrar las más similares
    
    if len(top_similar) >= 5:
        selected_similar = np.random.choice([score[0] for score in top_similar], 5, replace=False)
    else:
        selected_similar = [score[0] for score in top_similar[:5]]
    
    for movie_index in selected_similar:
        movie_title = movies_df_grouped['title'].iloc[movie_index]
        recommended_titles.add(movie_title)
    
    final_recommendations = list(recommended_titles)
    
    if len(final_recommendations) == 0:
        return "No hay suficientes recomendaciones disponibles."

    return final_recommendations

# Ejemplo de uso correcto
pelicula_recomendada = "Toy Story"  # Aquí simplemente asignas el nombre de la película directamente
print(recomendacion(pelicula_recomendada))


["Gone Fishin'", "Bosko's Parlor Pranks", 'Very Private Lesson', 'Mr. Magoo', 'Oliver & Company']


# 3. Este fue el mejor modelos 
* Da prioridad a las secuelas de peliculas

In [75]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
import re

def recomendacion(titulo, randomize=True):
    try:
        # Cargar datos
        movies_df = pd.read_parquet('../Datasets/movies_df.parquet')

        # Agrupar los géneros por título
        movies_df_grouped = movies_df.groupby('title').agg({
            'genre_name': lambda x: list(set(x))
        }).reset_index()

        titulo_lower = titulo.lower()
        
        # Verificar si la película existe
        pelicula = movies_df_grouped[movies_df_grouped['title'].str.lower() == titulo_lower]
        if pelicula.empty:
            return "Película no encontrada."
        
        # Vectorizar los títulos
        title_vectorizer = TfidfVectorizer(stop_words='english')
        title_vectors = title_vectorizer.fit_transform(movies_df_grouped['title'])
        
        # Codificar los géneros
        genre_encoder = MultiLabelBinarizer()
        genre_vectors = genre_encoder.fit_transform(movies_df_grouped['genre_name'])
        
        # Combinar vectores de título y género en una sola matriz
        combined_vectors = hstack([title_vectors, genre_vectors]).tocsr()
        
        # Obtener el índice de la película de entrada
        idx = movies_df_grouped.index[movies_df_grouped['title'].str.lower() == titulo_lower].tolist()[0]
        
        # Calcular la similitud de coseno
        input_vector = combined_vectors[idx]
        cosine_sim = cosine_similarity(input_vector, combined_vectors).flatten()
        
        # Excluir la película de entrada y ordenar por similitud
        sim_scores = sorted(list(enumerate(cosine_sim)), key=lambda x: x[1], reverse=True)
        sim_scores = [score for score in sim_scores if score[0] != idx]
        
        # Obtener los géneros de la película de entrada
        input_genres = set(movies_df_grouped['genre_name'].iloc[idx])
        
        # Filtrar por películas que coincidan en al menos 2 géneros
        filtered_movies = []
        for movie_index, sim_score in sim_scores:
            movie_genres = set(movies_df_grouped['genre_name'].iloc[movie_index])
            common_genres = input_genres.intersection(movie_genres)
            if len(common_genres) >= 2:
                filtered_movies.append(movie_index)
        
        # Obtener todas las secuelas
        sequel_titles = movies_df_grouped[movies_df_grouped['title'].str.contains(fr'{re.escape(titulo)} \d+', case=False, regex=True)]['title'].tolist()
        
        # Inicializar las listas de recomendaciones
        recommendations = []
        
        # Agregar secuelas primero
        if sequel_titles:
            recommendations.extend(sequel_titles)
        
        # Si hay espacio, agregar otras películas similares
        if len(recommendations) < 5:
            additional_movies = [movies_df_grouped['title'].iloc[movie_index] for movie_index in filtered_movies if movies_df_grouped['title'].iloc[movie_index] not in recommendations]
            if randomize:
                additional_movies = np.random.choice(additional_movies, min(5 - len(recommendations), len(additional_movies)), replace=False)
            else:
                additional_movies = additional_movies[:5 - len(recommendations)]
            
            recommendations.extend(additional_movies)
        
        # Garantizar que se devuelvan exactamente 5 recomendaciones
        if len(recommendations) > 5:
            recommendations = recommendations[:5]
        
        if len(recommendations) == 0:
            return "No hay suficientes recomendaciones disponibles."

        # Convertir los elementos de la lista a strings si son np.str_
        recommendations = [str(item) for item in recommendations]

        return f'Películas recomendadas: {recommendations}'

    except Exception as e:
        return {"error": str(e)}

# Ejemplo de uso
pelicula_recomendada = "toy story"
print(recomendacion(pelicula_recomendada))


Películas recomendadas: ['Toy Story 2', 'Toy Story 3', 'The Velveteen Rabbit', 'How to Train Your Dragon', "My Babysitter's a Vampire"]
